In [122]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


In [123]:
int_reactions= list()
i= -1
n= 0
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()
    n= int((len(csvFile)-1)/2)

    flt= float(csvFile.iloc[n,  2])
    
    if v == 1 and csvFile.iloc[n,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[n,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])


['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16a4colipa_e', 'EX_doxrbcn_e', 'EX_o2s_e', 'EX_xtsn_e', 'EX_pacald_e', 'EX_zn2_e', '12DGR120tipp', '12DGR140tipp', '12DGR141tipp', '12DGR160tipp', 'EX_pnto__R_e', '12DGR161tipp', '12DGR180tipp'

In [124]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    i3= int(((len(csvFile)-1)/2)+1)

    while i3< len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4< int(((len(csvFile)-1)/2)):
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2 lactose.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [125]:
i= 0
indecr_and_stable_above_zero= list()
diff_indecr_above_zero= list()
flat_above_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat above zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    
    my_series = d['Biomass'].squeeze()


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_above_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_above_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat_above_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)
    
    i= i+1

flat_above_zero
diff_indecr_above_zero
indecr_and_stable_above_zero


['ALAR',
 'CMtex',
 'CD2tpp',
 'COBALT2tpp',
 'DOXRBCNtex',
 'DURADx',
 'FBA3',
 'FHL',
 'FORtppi',
 'FUSAtex',
 'G6PDH2r',
 'GLCDpp',
 'GLCNt2rpp',
 'ICDHyr',
 'LCTStpp',
 'MINCYCtex',
 'MN2t3pp',
 'NOVBCNtex',
 'P5CR',
 'RFAMPtex',
 'PYAM5PO',
 'SPODM',
 'TTRCYCtex']

In [126]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat below zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    my_series = d['Biomass'].squeeze()
    

    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat_below_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


[]

In [127]:
i2= 0
mon_incr_above_zero= list()
mon_decr_above_zero= list()

if len(diff_indecr_above_zero)> 0:
    
    for r_id in diff_indecr_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_above_zero)
set2= set(mon_decr_above_zero)
set3= set(mon_incr_above_zero)
inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_above_zero


['MOAT',
 'MOAT2',
 'THRS',
 'ACLS',
 'THRD_L',
 '5DOAN',
 'NADS1',
 'K2L4Aabctex',
 'CU2tpp',
 'PRASCSi',
 'PMPK',
 'TMPK',
 'DASYN161',
 'ACONTa',
 'PPBNGS',
 'ANS',
 'ASPCT',
 'PRAGSr',
 'MTHFR2',
 '3OAS140',
 'DTMPK',
 'APG3PAT161',
 'ALAALAr',
 'IGPS',
 'AGPAT161',
 'ADSL1r',
 'PPND',
 'UAGCVT',
 'CYSTL',
 'HSK',
 'ACODA',
 'KDOPS',
 'DHAD2',
 'SADT2',
 'GLNS',
 'DHDPS',
 'PAPPT3',
 'ARGSS',
 'NH4tpp',
 'UAAGDS',
 'PRAIi',
 'SDPDS',
 'MCOATA',
 'HSTPT',
 'OCBT',
 'PRATPP',
 'UAMAGS',
 'UAPGR',
 'UAGAAT',
 'MPTG',
 'PSD160',
 'CTPS2',
 'HMBS',
 'UPP3S',
 'UPPDC1',
 'CHORM',
 'CYSS',
 'PGCD',
 'AGPAT160',
 'SHSL1',
 'BPNT',
 'OMPDC',
 'NNATr',
 'PItex',
 'AICART',
 'NDPK2',
 'SULR',
 'PSERT',
 'HCO3E',
 'TMDS',
 'HISTP',
 'AIRC2',
 'HISTD',
 'PSP_L',
 'PRAMPC',
 'NADK',
 'GLUTRR',
 'LPADSS',
 'SO4tex',
 'SERAT',
 'DM_5drib_c',
 'METAT',
 'PE160abcpp',
 'UMPK',
 'IPMD',
 'THDPS',
 'UAGDP',
 'PSD161',
 'ARGSL',
 'NH4tex',
 'UGMDDS',
 'APG3PAT160',
 'METS',
 'UAMAS',
 'USHD',
 'KDOCT2'

In [128]:
w= list()
w2= list()
i2=0

if len(inANDdecr_above_zero)> 0:
    
    while i2 < len(inANDdecr_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


['MOAT',
 'MOAT2',
 '5DOAN',
 'NADS1',
 'K2L4Aabctex',
 'CU2tpp',
 'PMPK',
 'TMPK',
 'DASYN161',
 'PPBNGS',
 'ANS',
 'MTHFR2',
 '3OAS140',
 'DTMPK',
 'APG3PAT161',
 'ALAALAr',
 'IGPS',
 'AGPAT161',
 'PPND',
 'UAGCVT',
 'CYSTL',
 'KDOPS',
 'PAPPT3',
 'UAAGDS',
 'PRAIi',
 'MCOATA',
 'HSTPT',
 'PRATPP',
 'UAMAGS',
 'UAPGR',
 'UAGAAT',
 'MPTG',
 'PSD160',
 'HMBS',
 'UPP3S',
 'UPPDC1',
 'AGPAT160',
 'SHSL1',
 'NNATr',
 'TMDS',
 'HISTP',
 'HISTD',
 'PRAMPC',
 'NADK',
 'GLUTRR',
 'LPADSS',
 'DM_5drib_c',
 'METAT',
 'PE160abcpp',
 'UAGDP',
 'PSD161',
 'UGMDDS',
 'APG3PAT160',
 'METS',
 'UAMAS',
 'USHD',
 'KDOCT2',
 'IGPDH',
 'ACPPAT160',
 'DASYN160',
 'GLUTRS',
 'A5PISO',
 'GF6PTA',
 'G1PACT',
 'PSSA160',
 'UHGADA',
 'K2L4Aabcpp',
 'NDPK4',
 'TDSK',
 'ATPPRT',
 'ACCOAC',
 'U23GAAT',
 'PE161abcpp',
 'PRMICI',
 '3OAR140',
 'ACPPAT161',
 'DHFR',
 'PSSA161',
 'ANPRT',
 'FCLT',
 'HSST',
 'MCTP1App',
 'UDCPDP',
 'G1SAT',
 'UAGPT3',
 'KDOPP',
 'IG3PS']

In [129]:
set1= set(inANDdecr_above_zero)
set2= set(w)
set3= set1-set2
inANDdecr_above_zero= list(set3)

mon_decr_above_zero= list(mon_decr_above_zero) + list(w)


set1= set(inANDdecr_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_above_zero= list(set3)

mon_incr_above_zero= list(mon_incr_above_zero) + list(w2)


In [130]:
i2= 0
mon_incr_below_zero= list()
mon_decr_below_zero= list()

if len(diff_indecr_below_zero)> 0:
    
    for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero)
set2= set(mon_decr_below_zero)
set3= set(mon_incr_below_zero)
inANDdecr_below_zero= list(set1-set2-set3)
inANDdecr_below_zero


[]

In [131]:
w= list()
w2= list()
i2=0

if len(inANDdecr_below_zero)> 0:
    
    while i2 < len(inANDdecr_below_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


[]

In [132]:
set1= set(inANDdecr_below_zero)
set2= set(w)
set3= set1-set2
inANDdecr_below_zero= list(set3)

mon_decr_below_zero= list(mon_decr_below_zero) + list(w)


set1= set(inANDdecr_below_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_below_zero= list(set3)

mon_incr_below_zero= list(mon_incr_below_zero) + list(w2)


In [133]:
i2= 0
mon_incr_and_stable_above_zero= list()
mon_decr_and_stable_above_zero= list()

if len(indecr_and_stable_above_zero)> 0:
    
    for r_id in indecr_and_stable_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_above_zero)
inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_above_zero


['P5CR', 'COBALT2tpp', 'G6PDH2r', 'ICDHyr', 'ALAR']

In [134]:
w= list()
w2= list()
i2=0

if len(inANDdecr_and_stable_above_zero)> 0:
    
    while i2 < len(inANDdecr_and_stable_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


['COBALT2tpp', 'ALAR']

In [135]:
set1= set(inANDdecr_and_stable_above_zero)
set2= set(w)
set3= set1-set2
inANDdecr_and_stable_above_zero= list(set3)

mon_decr_and_stable_above_zero= list(mon_decr_and_stable_above_zero) + list(w)


set1= set(inANDdecr_and_stable_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_and_stable_above_zero= list(set3)

mon_incr_and_stable_above_zero= list(mon_incr_and_stable_above_zero) + list(w2)


In [136]:
i2= 0
mon_incr_and_stable_below_zero= list()
mon_decr_and_stable_below_zero= list()

if len(indecr_and_stable_below_zero)> 0:
    
    for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero.append(r_id)

        i2= i2 + 1

set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_below_zero)
set3= set(mon_incr_and_stable_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


[]

In [137]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero):       #this code finds reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

            
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)

    
        i2= i2+1

w3


[]

In [138]:
set1= set(inANDdecr_and_stable_below_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero= list(set4)

mon_decr_and_stable_below_zero= list(mon_decr_and_stable_below_zero) + list(w3)
mon_incr_and_stable_below_zero= list(mon_incr_and_stable_below_zero) + list(w4)


In [139]:
i= 0
over_below_zero_and_under_above_zero= list()
under_below_zero_and_over_above_zero= list()
over_below_zero_and_over_above_zero= list()
under_below_zero_and_under_above_zero= list()
r_id = a.reactions[0]

for r_id in a.reactions:    #separates reactions based on whether they have less than 30 or not values both in the above and the below zero interval
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < int(((len(csvFile)-1)/2)+1):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= int((len(csvFile)-1)/2)
    l4= list()
    
    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if (len(csvFile2)>30 or len(csvFile2)==30) and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
        
        over_below_zero_and_under_above_zero.append(r_id)
        
    if len(csvFile2)<30 and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        under_below_zero_and_over_above_zero.append(r_id)
    
    if (len(csvFile2)>30 or len(csvFile2)==30) and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        over_below_zero_and_over_above_zero.append(r_id)
        
    if len(csvFile2)<30 and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
    
        under_below_zero_and_under_above_zero.append(r_id)
    
    
    i= i+1
    
over_below_zero_and_under_above_zero


['EX_cmp_e',
 'EX_cpgn_e',
 'EX_cpgn_un_e',
 'EX_csn_e',
 'EX_cu_e',
 'EX_glcr_e',
 'EX_glcur_e',
 'EX_glcur1p_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_gtp_e',
 'EX_h2_e',
 'EX_23camp_e',
 'EX_23ccmp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_23dappa_e',
 'EX_26dap__M_e',
 'EX_2ddglcn_e',
 'EX_hdca_e',
 'EX_3amp_e',
 'EX_hdcea_e',
 'EX_3cmp_e',
 'EX_cynt_e',
 'EX_3gmp_e',
 'EX_3hcinnm_e',
 'EX_3hpppn_e',
 'EX_3ump_e',
 'EX_LalaDglu_e',
 'EX_cytd_e',
 'EX_LalaLglu_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_isetac_e',
 'EX_acgal1p_e',
 'EX_acgam_e',
 'EX_acgam1p_e',
 'EX_acmana_e',
 'EX_acmum_e',
 'EX_acnam_e',
 'EX_dca_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_lyx__L_e',
 'EX_mal__D_e',
 'EX_ddca_e',
 'EX_malt_e',
 'EX_all__D_e',
 'EX_dgmp_e',
 'EX_man_e',
 'EX_man6p_e',
 'EX_manglyc_e',
 'EX_amp_e',
 'EX_arab__L_e',
 'EX_arbtn_e',
 'EX_melib_e',
 'EX_arbtn_fe3_e',
 'EX_dgsn_e',
 'EX_ascb__L_e',
 'EX_met__L_e',
 'EX_mets

In [140]:
i= 0
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_diff_indecr_above_zero= list()
rest_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
flat_below_zero_and_flat_above_zero= list()
diff_indecr_below_zero_and_flat_above_zero= list()
flat_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
flat_below_zero_and_one_to_three_data_point_above_zero= list()
diff_indecr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(over_below_zero_and_under_above_zero)> 0:

    for r_id in over_below_zero_and_under_above_zero:    #separates over_below_zero_and_under_above_zero reactions in more specific subcategories
    
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and len(csvFile3)<4:
            indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and (count2> 3 and count2< len(csvFile3)-4):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 4:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and len(csvFile3)<4:
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 4:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 4):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
        elif count< 0.05*len(csvFile2) and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 3 and count2< len(csvFile3)-4):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 4:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif count< 0.05*len(csvFile2) and (count2> len(csvFile3)-4):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and (count2> 2 and len(csvFile3) < 4):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [141]:
i=0
one_to_three_data_point_below_zero_and_diff_indecr_above_zero= list()
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_flat_above_zero= list()

if len(under_below_zero_and_over_above_zero)> 0:

    for r_id in under_below_zero_and_over_above_zero:    #separates under_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 4 and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<4 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

            
    
        elif len(csvFile2)<4 and count2< 0.05*len(csvFile3):
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 4 and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
        
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [142]:
i=0

if len(over_below_zero_and_over_above_zero)> 0:

    for r_id in over_below_zero_and_over_above_zero:    #separates over_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count<0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ASPt2_2pp',
 'CMtpp',
 'DAAD',
 'DOXRBCNtpp',
 'FADRx',
 'FADRx2',
 'FLDR2',
 'FE2tpp',
 'FORt2pp',
 'FUSAtpp',
 'GLCabcpp',
 'GLCt2pp',
 'ASPT',
 'GALabcpp',
 'GUAt2pp',
 'IDOND2',
 'HPYRRy',
 'GND',
 'MINCYCtpp',
 'MN2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'PGI',
 'PGL',
 'RFAMPtpp',
 'PYK',
 'SUCOAS',
 'TALA',
 'THD2pp',
 'THRA2',
 'TTRCYCtpp',
 'XANt2pp',
 'PPC',
 'DURAD']

In [143]:
i=0
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero= list()

if len(under_below_zero_and_under_above_zero)> 0:
    
    for r_id in under_below_zero_and_under_above_zero:    #separates under_below_zero_and_under_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and len(csvFile3)<4:
            one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

        elif len(csvFile2)<4 and (count2> 3 and count2< len(csvFile3)-4):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 3 and count2< len(csvFile3)-4):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 4 and (count2> 3 and count2< len(csvFile3)-4):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 2 and len(csvFile3) < 4):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 1 and len(csvFile3) < 3):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 2 and len(csvFile3) < 4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<4 and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<4 and (count2> 2 and len(csvFile3) < 4):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<4 and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)

        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 2 and len(csvFile3) < 4):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> 2 and len(csvFile3) < 4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            flat_below_zero_and_flat_above_zero.append(r_id)

        elif (count> 1 and len(csvFile2) < 3) and (count2> 2 and len(csvFile3) < 4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and count2< 4:
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 4 and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and count2< 4:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 4 and count2< 4:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and count2< 4:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and count2< 4:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 4:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ASPt2_2pp',
 'CMtpp',
 'DAAD',
 'DOXRBCNtpp',
 'FADRx',
 'FADRx2',
 'FLDR2',
 'FE2tpp',
 'FORt2pp',
 'FUSAtpp',
 'GLCabcpp',
 'GLCt2pp',
 'ASPT',
 'GALabcpp',
 'GUAt2pp',
 'IDOND2',
 'HPYRRy',
 'GND',
 'MINCYCtpp',
 'MN2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'PGI',
 'PGL',
 'RFAMPtpp',
 'PYK',
 'SUCOAS',
 'TALA',
 'THD2pp',
 'THRA2',
 'TTRCYCtpp',
 'XANt2pp',
 'PPC',
 'DURAD']

In [144]:
flat_below_zero_and_flat_above_zero
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero
indecr_and_stable_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_diff_indecr_above_zero
diff_indecr_below_zero_and_indecr_and_stable_above_zero
diff_indecr_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_diff_indecr_above_zero
flat_below_zero_and_indecr_and_stable_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_flat_above_zero


indecr_and_stable_below_zero_and_indecr_and_stable_above_zero
indecr_and_stable_below_zero_and_flat_above_zero


['GLCptspp', 'Htex', 'LACZ', 'NADH17pp', 'PTAr']

In [145]:
len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+len(indecr_and_stable_below_zero_and_flat_above_zero)+len(flat_below_zero_and_flat_above_zero)+len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_flat_above_zero)+len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_diff_indecr_above_zero)+len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_diff_indecr_above_zero)+len(flat_below_zero_and_indecr_and_stable_above_zero)+len(flat_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_flat_above_zero)+len(one)+len(zero)+len(one_at_zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(diff_indecr_above_zero)+len(diff_indecr_below_zero)


2726

In [146]:
lt= list(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+list(indecr_and_stable_below_zero_and_flat_above_zero)+list(flat_below_zero_and_flat_above_zero)+list(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_flat_above_zero)+list(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_diff_indecr_above_zero)+list(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+list(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_diff_indecr_above_zero)+list(flat_below_zero_and_indecr_and_stable_above_zero)+list(flat_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_flat_above_zero)+list(one)+list(zero)+list(one_at_zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(diff_indecr_above_zero)+list(diff_indecr_below_zero)


In [147]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [148]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [149]:
i2= 0
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()

if len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero:                    #this code finds which of the indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero

set1= set(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set3= set(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero


['EX_orot_e', 'ASPTA']

In [150]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
    
        i2= i2+1

w3


[]

In [151]:
set1= set(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set4)

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [152]:
i2= 0
indecr_and_stable_below_zero_and_mon_incr_above_zero= list()
indecr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_diff_indecr_above_zero:                  #this code finds which of the indecr_and_stable_below_zero_and_diff_indecr_above_zero reactions have an increasing trend and which a decreasing trend above zero
    
        r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                indecr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                indecr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_diff_indecr_above_zero)
set2= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
indecr_and_stable_below_zero_and_inANDdecr_above_zero


['GARFT', 'PIt2rpp']

In [153]:
w3= list()
w4= list()
i2=0

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(indecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing above zero
    
        i= int((len(csvFile)-1)/2)
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        
        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)
        
            
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)

    
        i2= i2+1

w3


['GARFT']

In [154]:
set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set4)

indecr_and_stable_below_zero_and_mon_decr_above_zero= list(indecr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
indecr_and_stable_below_zero_and_mon_incr_above_zero= list(indecr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [155]:
i2= 0
decr_and_stable_below_zero_and_mon_decr_above_zero= list()
incr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_mon_decr_above_zero:                #this code finds which of the indecr_and_stable_below_zero_and_mon_decr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_decr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['FBP', 'ACOAD6f', 'CITL', 'ACOAD5f', 'GLYCDx', 'GLUDy']

In [156]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_decr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [157]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set4)

decr_and_stable_below_zero_and_mon_decr_above_zero= list(decr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_decr_above_zero= list(incr_and_stable_below_zero_and_mon_decr_above_zero) + list(w4)


In [158]:
i2= 0
decr_and_stable_below_zero_and_mon_incr_above_zero= list()
incr_and_stable_below_zero_and_mon_incr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_incr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_mon_incr_above_zero:              #this code finds which of the indecr_and_stable_below_zero_and_mon_incr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_incr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_incr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero


[]

In [159]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_incr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [160]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set4)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [161]:
i2= 0
decr_and_stable_below_zero_and_inANDdecr_above_zero= list()
incr_and_stable_below_zero_and_inANDdecr_above_zero= list()

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_inANDdecr_above_zero:           #this code finds which of the indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)


        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(decr_and_stable_below_zero_and_inANDdecr_above_zero)
set3= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [162]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [163]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set4)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [164]:
i2= 0
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list()

if len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)
set2= set(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)
set3= set(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero


['FE2t3pp']

In [165]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['FE2t3pp']

In [166]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero) + list(w4)


In [167]:
i2= 0
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list()

if len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_diff_indecr_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)
set2= set(one_to_three_data_point_below_zero_and_mon_decr_above_zero)
set3= set(one_to_three_data_point_below_zero_and_mon_incr_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero


[]

In [168]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [169]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set4)

one_to_three_data_point_below_zero_and_mon_decr_above_zero= list(one_to_three_data_point_below_zero_and_mon_decr_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list(one_to_three_data_point_below_zero_and_mon_incr_above_zero) + list(w4)


In [170]:
i2= 0
diff_indecr_below_zero_and_mon_incr_above_zero= list()
diff_indecr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_diff_indecr_above_zero:
    
        r_id = diff_indecr_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                diff_indecr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                diff_indecr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_diff_indecr_above_zero)
set2= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set3= set(diff_indecr_below_zero_and_mon_incr_above_zero)
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
diff_indecr_below_zero_and_inANDdecr_above_zero


['4PCPpp',
 'DALAt2pp',
 'AACPS3',
 'ECOAH6',
 'ECOAH1',
 'ENO',
 'FEOXAMR3',
 'AGM3PA',
 'MLTGY1pp',
 'TKT1',
 'FE3HOXR2',
 '4PCP',
 'AGM4PCPpp',
 'MLTGY2pp',
 '3PEPTabcpp',
 'MTHFD',
 'SPMDabcpp',
 '4PEPTabcpp',
 'GPDDA4pp',
 'AACPS4',
 'NDPK3',
 'MLDCP1Bpp',
 'AM4PA',
 'FE3HOXR3',
 'CPGNR2',
 'DURIPP',
 'AM4PCP',
 'FECRMR2',
 'FEOXAMR2',
 'TMDK1',
 'XPPT',
 'CPPPGO',
 'ARBTNR2',
 'TKT2',
 'CYTK1',
 'CPGNR3',
 'ECOAH2',
 'ECOAH4',
 'NTD1',
 'ALAALAabcpp',
 'CBMKr',
 'CYSabcpp',
 'FEENTERR3',
 'GHMT2r',
 'AACPS2',
 'MDDCP1pp',
 'ETHAt2pp',
 'FEENTERR2',
 'ECOAH3',
 'MTHFC',
 'AGM3Pt2pp',
 'AGM4PCP',
 'MDDEP3pp',
 'AGMt2pp',
 'ARBTNR3',
 'FECRMR3',
 'MLDEP2pp',
 'AGM3PH',
 'MLDCP2Bpp',
 'AACPS7',
 'AGM3PApp']

In [171]:
w3= list()
w4= list()
i2=0

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(diff_indecr_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['4PCPpp',
 'DALAt2pp',
 'AACPS3',
 'FEOXAMR3',
 'AGM3PA',
 'MLTGY1pp',
 'FE3HOXR2',
 '4PCP',
 'AGM4PCPpp',
 'MLTGY2pp',
 '3PEPTabcpp',
 'SPMDabcpp',
 '4PEPTabcpp',
 'GPDDA4pp',
 'AACPS4',
 'NDPK3',
 'MLDCP1Bpp',
 'AM4PA',
 'FE3HOXR3',
 'CPGNR2',
 'DURIPP',
 'AM4PCP',
 'FECRMR2',
 'FEOXAMR2',
 'TMDK1',
 'XPPT',
 'CPPPGO',
 'ARBTNR2',
 'CYTK1',
 'CPGNR3',
 'NTD1',
 'ALAALAabcpp',
 'CYSabcpp',
 'FEENTERR3',
 'AACPS2',
 'MDDCP1pp',
 'ETHAt2pp',
 'FEENTERR2',
 'AGM3Pt2pp',
 'AGM4PCP',
 'MDDEP3pp',
 'AGMt2pp',
 'ARBTNR3',
 'FECRMR3',
 'MLDEP2pp',
 'AGM3PH',
 'MLDCP2Bpp',
 'AACPS7',
 'AGM3PApp']

In [172]:
w4

['ENO']

In [173]:
w4

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set4)

diff_indecr_below_zero_and_mon_decr_above_zero= list(diff_indecr_below_zero_and_mon_decr_above_zero) + list(w3)
diff_indecr_below_zero_and_mon_incr_above_zero= list(diff_indecr_below_zero_and_mon_incr_above_zero) + list(w4)


In [174]:
i2= 0
mon_decr_below_zero_and_mon_decr_above_zero= list()
mon_incr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_decr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_above_zero)
set3= set(mon_incr_below_zero_and_mon_decr_above_zero)
inANDdecr_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_decr_above_zero


['ACOAD4f',
 'ACOAD2f',
 'ACOAD1f',
 'EX_ura_e',
 'EX_val__L_e',
 'EX_quin_e',
 'ACOAD3f',
 'PGM',
 'ME2',
 'RPI']

In [175]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_decr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

        
        
w3


[]

In [176]:
w4

[]

In [177]:
w4

set1= set(inANDdecr_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_mon_decr_above_zero= list(set4)

mon_decr_below_zero_and_mon_decr_above_zero= list(mon_decr_below_zero_and_mon_decr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_decr_above_zero= list(mon_incr_below_zero_and_mon_decr_above_zero) + list(w4)


In [178]:
i2= 0
mon_decr_below_zero_and_mon_incr_above_zero= list()
mon_incr_below_zero_and_mon_incr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_incr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_incr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_incr_above_zero)
set2= set(mon_decr_below_zero_and_mon_incr_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_above_zero)
inANDdecr_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_incr_above_zero


[]

In [179]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_incr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [180]:
w4

set1= set(inANDdecr_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_mon_incr_above_zero= list(set4)

mon_decr_below_zero_and_mon_incr_above_zero= list(mon_decr_below_zero_and_mon_incr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_above_zero= list(mon_incr_below_zero_and_mon_incr_above_zero) + list(w4)


In [181]:
i2= 0
mon_decr_below_zero_and_inANDdecr_above_zero= list()
mon_incr_below_zero_and_inANDdecr_above_zero= list()

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_inANDdecr_above_zero:
    
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_inANDdecr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(mon_decr_below_zero_and_inANDdecr_above_zero)
set3= set(mon_incr_below_zero_and_inANDdecr_above_zero)
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_above_zero


[]

In [182]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [183]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set4)

mon_decr_below_zero_and_inANDdecr_above_zero= list(mon_decr_below_zero_and_inANDdecr_above_zero) + list(w3)
mon_incr_below_zero_and_inANDdecr_above_zero= list(mon_incr_below_zero_and_inANDdecr_above_zero) + list(w4)


In [184]:
i2= 0
mon_decr_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_below_zero_and_indecr_and_stable_above_zero= list()

if len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = diff_indecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_indecr_and_stable_above_zero


[]

In [185]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [186]:
w4

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set4)

mon_decr_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [187]:
i2= 0
inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [188]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [189]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [190]:
i2= 0
mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [191]:
w3= list()
w4= list()
i2=0

if len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [192]:
w4

set1= set(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [193]:
i2= 0
mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero


['FUM', 'ECOAH5', 'RPE', 'TPI', 'MDH', 'GNK', 'ECOAH7', 'ICL', 'AKGDH']

In [194]:
w3= list()
w4= list()
i2=0

if len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['GNK', 'ECOAH7', 'ICL']

In [195]:
w4

['ECOAH7']

In [196]:
w4

set1= set(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [197]:
i2= 0
mon_decr_below_zero_and_one_to_three_data_point_above_zero= list()
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_one_to_three_data_point_above_zero:
    
        r_id = diff_indecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(mon_decr_below_zero_and_one_to_three_data_point_above_zero)
set3= set(mon_incr_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero


['KARA1',
 'PHETA1',
 'EX_metsox_S__L_e',
 'EX_skm_e',
 'EX_isetac_e',
 'EX_met__L_e',
 'EX_nh4_e',
 'HSDy',
 'IPPMIa',
 'AGPR',
 'EX_pi_e',
 'ILETA',
 'AIRC3',
 'EX_26dap__M_e',
 'IMPC',
 'EX_metsox_R__L_e',
 'ORPT',
 'DHORTS',
 'ASAD',
 'SDPTA',
 'G3PD2',
 'TYRTA',
 'EX_sulfac_e',
 'PGAMT',
 'EX_so4_e',
 'ACOTA',
 'IPPMIb',
 'EX_mso3_e']

In [198]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [199]:
w4

set1= set(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set4)

mon_decr_below_zero_and_one_to_three_data_point_above_zero= list(mon_decr_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list(mon_incr_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [200]:
i2= 0
flat_below_zero_and_mon_decr_above_zero= list()
flat_below_zero_and_mon_incr_above_zero= list()

if len(flat_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in flat_below_zero_and_diff_indecr_above_zero:
    
        r_id = flat_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_diff_indecr_above_zero)
set2= set(flat_below_zero_and_mon_decr_above_zero)
set3= set(flat_below_zero_and_mon_incr_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [201]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [202]:
w4

set1= set(flat_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
flat_below_zero_and_inANDdecr_above_zero= list(set4)

flat_below_zero_and_mon_decr_above_zero= list(flat_below_zero_and_mon_decr_above_zero) + list(w3)
flat_below_zero_and_mon_incr_above_zero= list(flat_below_zero_and_mon_incr_above_zero) + list(w4)


In [203]:
i2= 0
flat_below_zero_and_mon_decr_and_stable_above_zero= list()
flat_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(flat_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in flat_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = flat_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_indecr_and_stable_above_zero)
set2= set(flat_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(flat_below_zero_and_mon_incr_and_stable_above_zero)
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [204]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [205]:
w4

set1= set(flat_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

flat_below_zero_and_mon_decr_and_stable_above_zero= list(flat_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
flat_below_zero_and_mon_incr_and_stable_above_zero= list(flat_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [206]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [207]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [208]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_flat_above_zero= list(set4)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [209]:
i2= 0
mon_decr_and_stable_below_zero_and_flat_above_zero= list()
mon_incr_and_stable_below_zero_and_flat_above_zero= list()

if len(indecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_flat_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_flat_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_flat_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_flat_above_zero)
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_flat_above_zero


[]

In [210]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [211]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set4)

mon_decr_and_stable_below_zero_and_flat_above_zero= list(mon_decr_and_stable_below_zero_and_flat_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_flat_above_zero= list(mon_incr_and_stable_below_zero_and_flat_above_zero) + list(w4)


In [212]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [213]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [214]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_flat_above_zero= list(set4)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [215]:
i2= 0
mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()

if len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['FLDR2', 'GLCt2pp']

In [216]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [217]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set4)

mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [218]:
i2= 0
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [219]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [220]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [221]:
i2= 0
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [222]:
w3= list()
w4= list()
i2=0

if len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [223]:
w4

set1= set(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [224]:
i2= 0
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


['PGI', 'PGL', 'FADRx', 'GND', 'PPC']

In [225]:
w3= list()
w4= list()
i2=0

if len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [226]:
w4

set1= set(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [227]:
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero

decr_and_stable_below_zero_and_mon_decr_above_zero
incr_and_stable_below_zero_and_mon_decr_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero

decr_and_stable_below_zero_and_mon_incr_above_zero
incr_and_stable_below_zero_and_mon_incr_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero

decr_and_stable_below_zero_and_inANDdecr_above_zero
incr_and_stable_below_zero_and_inANDdecr_above_zero
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero

one_to_three_data_point_below_zero_and_mon_decr_above_zero
one_to_three_data_point_below_zero_and_mon_incr_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_above_zero
mon_decr_below_zero_and_mon_decr_above_zero
mon_incr_below_zero_and_mon_decr_above_zero

inANDdecr_below_zero_and_mon_incr_above_zero
mon_decr_below_zero_and_mon_incr_above_zero
mon_incr_below_zero_and_mon_incr_above_zero

inANDdecr_below_zero_and_inANDdecr_above_zero
mon_decr_below_zero_and_inANDdecr_above_zero
mon_incr_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero

mon_decr_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_below_zero_and_one_to_three_data_point_above_zero
mon_decr_below_zero_and_one_to_three_data_point_above_zero
mon_incr_below_zero_and_one_to_three_data_point_above_zero

flat_below_zero_and_inANDdecr_above_zero
flat_below_zero_and_mon_decr_above_zero
flat_below_zero_and_mon_incr_above_zero

flat_below_zero_and_inANDdecr_and_stable_above_zero
flat_below_zero_and_mon_decr_and_stable_above_zero
flat_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_and_stable_below_zero_and_flat_above_zero
mon_decr_and_stable_below_zero_and_flat_above_zero
mon_incr_and_stable_below_zero_and_flat_above_zero

inANDdecr_below_zero_and_flat_above_zero
mon_decr_below_zero_and_flat_above_zero
mon_incr_below_zero_and_flat_above_zero

inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

flat_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero


['EX_cpgn_un_e',
 'EX_cu_e',
 'EX_gtp_e',
 'EX_acgal1p_e',
 'EX_arbtn_e',
 'EX_minohp_e',
 'EX_dopa_e',
 'EX_xmp_e',
 'EX_peamn_e',
 'EX_tym_e',
 'EX_tyrp_e',
 'EX_fe3hox_un_e',
 'EX_fecrm_un_e',
 'EX_feoxam_un_e',
 'ACt2rpp',
 'ALAt2pp_copy2',
 'ALDD19xr_copy1',
 'AMALT2',
 'AMALT3',
 'AMALT4',
 'CAt6pp',
 'CRNCAL2',
 'CRNt2rpp',
 'CTBTCAL2',
 'CU1Opp',
 'DHNPA2r_copy1',
 'DKGLCNR2x',
 'DMPPS',
 'DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'DSBAO1',
 'DSBAO2',
 'GLCtex_copy1',
 'GLYCLTt2rpp',
 'HYD1pp',
 'HYD2pp',
 'HYD3pp',
 'IPDPS',
 'HPYRI',
 'GLYt2pp_copy2',
 'MECDPDH5',
 'NO3R2pp',
 'NHFRBO',
 'NMNt7pp',
 'NO3R1pp',
 'OCTNLL',
 'PDX5PO2',
 'PDX5POi',
 'PROt2rpp',
 'QMO2',
 'QMO3',
 'SERt2rpp',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp',
 'TREHpp',
 'URAt2pp_copy2',
 'NPHS',
 'DARBt2rpp_copy1']

In [228]:
"FE3HOXR3" in mon_incr_below_zero_and_mon_decr_above_zero


True

In [229]:
mon_incr_below_zero_and_mon_decr_above_zero

['PPPGO3',
 'EX_indole_e',
 '2AGPEAT120',
 '2AGPEAT140',
 '2AGPEAT141',
 '2AGPEAT160',
 '2AGPEAT161',
 '2AGPEAT180',
 '2AGPEAT181',
 '2AGPGAT120',
 '2AGPGAT140',
 '2AGPGAT141',
 '2AGPGAT160',
 '2AGPGAT161',
 '2AGPGAT180',
 '2AGPGAT181',
 'AACPS5',
 'AACPS6',
 'AACPS1',
 'ACOATA',
 'AGM4PA',
 'CDAPPA181',
 'APH120',
 'CLPNH120pp',
 'CLPNH140pp',
 'CLPNH141pp',
 'CLPNH160pp',
 'APH140',
 'CLPNH161pp',
 'CLPNH180pp',
 'CLPNH181pp',
 'APH141',
 'APH160',
 'APH161',
 'CMPN',
 'APH180',
 'CDAPPA120',
 'APH181',
 'CDAPPA140',
 'CDAPPA141',
 'CDAPPA160',
 'CDAPPA161',
 'CDAPPA180',
 'DADK',
 'DCTPD',
 'CYSDS',
 'CYStpp',
 'DGK1',
 'CYTD',
 'CYTDH',
 'CYTDK2',
 'CYTK2',
 'ARBt2rpp',
 'ARGAGMt7pp',
 'DURIK1',
 'ENTERES',
 'ENTERES2',
 'FA100ACPHi',
 'FA80ACPHi',
 'FACOAE100',
 'FACOAE120',
 'FACOAE140',
 'FACOAE141',
 'FACOAE160',
 'FACOAE161',
 'FACOAE180',
 'FACOAE181',
 'FACOAE80',
 'ASNS2',
 'FTHFD',
 'G3PEabcpp',
 'G3PGabcpp',
 'G3PT',
 'G6PDA',
 'GLUDC',
 'GDPDPK',
 'GDPMNH',
 'GDPTPDP',
 

In [230]:
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero

['EX_csn_e',
 'EX_h2_e',
 'EX_cynt_e',
 'EX_ppt_e',
 'EX_ethso3_e',
 '2DGULRGy',
 '2DGULRx',
 '2DGULRy',
 'ACOLIPAabctex',
 'CHLabcpp',
 'BETALDHx',
 'BSORy',
 'COLIPAabctex',
 'CU1abcpp',
 'DKGLCNR2y',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GLYBabcpp',
 'HG2abcpp',
 'NODOy',
 'NO3R1bpp',
 'NO3R2bpp',
 'OMMBLHX3',
 'OMPHHX3',
 'OPHHX3',
 'XYLabcpp',
 '2DGLCNRy',
 'DOGULND']

In [231]:
"ECOAH7" in mon_incr_below_zero_and_mon_decr_and_stable_above_zero
"ECOAH7" in mon_incr_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_below_zero_and_mon_incr_and_stable_above_zero.remove("ECOAH7")


In [232]:
len(flat_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ len(flat_below_zero_and_one_to_three_data_point_above_zero)+ len(one_at_zero)+len(one)+len(zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(diff_indecr_above_zero)+len(diff_indecr_below_zero)+len(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(decr_and_stable_below_zero_and_mon_incr_above_zero)+len(incr_and_stable_below_zero_and_mon_incr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+len(decr_and_stable_below_zero_and_inANDdecr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_above_zero)+len(mon_decr_below_zero_and_mon_decr_above_zero)+len(mon_incr_below_zero_and_mon_decr_above_zero)+len(inANDdecr_below_zero_and_mon_incr_above_zero)+len(mon_decr_below_zero_and_mon_incr_above_zero)+len(mon_incr_below_zero_and_mon_incr_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_above_zero)+len(mon_decr_below_zero_and_inANDdecr_above_zero)+len(mon_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_inANDdecr_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_mon_incr_above_zero)+len(flat_below_zero_and_inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_mon_decr_and_stable_above_zero)+len(flat_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_flat_above_zero)+len(mon_decr_and_stable_below_zero_and_flat_above_zero)+len(mon_incr_and_stable_below_zero_and_flat_above_zero)+len(inANDdecr_below_zero_and_flat_above_zero)+len(mon_decr_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)


2726

In [233]:
lt= list(flat_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ list(flat_below_zero_and_one_to_three_data_point_above_zero)+ list(one_at_zero)+list(one)+list(zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(diff_indecr_above_zero)+list(diff_indecr_below_zero)+list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+list(decr_and_stable_below_zero_and_mon_incr_above_zero)+list(incr_and_stable_below_zero_and_mon_incr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+list(decr_and_stable_below_zero_and_inANDdecr_above_zero)+list(incr_and_stable_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+list(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_above_zero)+list(mon_decr_below_zero_and_mon_decr_above_zero)+list(mon_incr_below_zero_and_mon_decr_above_zero)+list(inANDdecr_below_zero_and_mon_incr_above_zero)+list(mon_decr_below_zero_and_mon_incr_above_zero)+list(mon_incr_below_zero_and_mon_incr_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_above_zero)+list(mon_decr_below_zero_and_inANDdecr_above_zero)+list(mon_incr_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_inANDdecr_above_zero)+list(flat_below_zero_and_mon_decr_above_zero)+list(flat_below_zero_and_mon_incr_above_zero)+list(flat_below_zero_and_inANDdecr_and_stable_above_zero)+list(flat_below_zero_and_mon_decr_and_stable_above_zero)+list(flat_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_flat_above_zero)+list(mon_decr_and_stable_below_zero_and_flat_above_zero)+list(mon_incr_and_stable_below_zero_and_flat_above_zero)+list(inANDdecr_below_zero_and_flat_above_zero)+list(mon_decr_below_zero_and_flat_above_zero)+list(mon_incr_below_zero_and_flat_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)


In [234]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [235]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [236]:
len(flat_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ len(flat_below_zero_and_one_to_three_data_point_above_zero)+ len(one_at_zero)+len(one)+len(zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)+len(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(decr_and_stable_below_zero_and_mon_incr_above_zero)+len(incr_and_stable_below_zero_and_mon_incr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+len(decr_and_stable_below_zero_and_inANDdecr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_above_zero)+len(mon_decr_below_zero_and_mon_decr_above_zero)+len(mon_incr_below_zero_and_mon_decr_above_zero)+len(inANDdecr_below_zero_and_mon_incr_above_zero)+len(mon_decr_below_zero_and_mon_incr_above_zero)+len(mon_incr_below_zero_and_mon_incr_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_above_zero)+len(mon_decr_below_zero_and_inANDdecr_above_zero)+len(mon_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_inANDdecr_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_mon_incr_above_zero)+len(flat_below_zero_and_inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_mon_decr_and_stable_above_zero)+len(flat_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_flat_above_zero)+len(mon_decr_and_stable_below_zero_and_flat_above_zero)+len(mon_incr_and_stable_below_zero_and_flat_above_zero)+len(inANDdecr_below_zero_and_flat_above_zero)+len(mon_decr_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)


2726

In [237]:
len(diff_indecr_above_zero)+len(diff_indecr_below_zero)


609

In [238]:
len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)


609